In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pickle
import numpy as np
from keras.models import load_model
import json
import random
import re
model = load_model('model/models.keras')
stop_words = set(stopwords.words("indonesian"))
stemmer = PorterStemmer()
intents = json.loads(open('model/intents.json').read())
words = pickle.load(open('model/texts.pkl','rb'))
classes = pickle.load(open('model/labels.pkl','rb'))
with open('model/baku.json') as f:
    normalization_data = json.load(f)

In [3]:
def normalize_word(word):
    return normalization_data.get(word, word)

# NLP
def nlp_steps(sentence):
    # Case Folding
    sentence = sentence.lower()
    # Normalization
    sentence_words = [normalize_word(sentence) for word in sentence]
    # Tokenization
    sentence_words = nltk.word_tokenize(sentence)
    # Menghapus karakter spesial
    sentence_words = [re.sub(r'[^\w\s]', '', word) for word in sentence_words]
    # Menghapus angka
    sentence_words = [re.sub(r'\d+', '', word) for word in sentence_words]
    # Filtering (menghapus stop words)
    sentence_words = [word for word in sentence_words if word not in stop_words]
    # Stemming
    sentence_words = [stemmer.stem(word) for word in sentence_words]
    return sentence_words

# Bag of Words
def bow(sentence, words, show_details=True):
    sentence_words = nlp_steps(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

# Untuk memprediksi maksud/keinginan dari kata - kata inputan pengguna dengan model yang sudah dibuat
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    if results:
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        return return_list
    else:
        return [{"intent": "fallback", "probability": "1.0"}]

# Untuk memilih jawaban yang sesuai dengan dataset
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# Untuk menampilkan hasil getResponse    
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [4]:
chatbot_response("Saya sudah batuk selama 3 hari")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step


'1. Madu: Gunakan madu untuk mengobati batuk, campurkan 2 sendok teh dengan air hangat atau teh herbal. Minum ramuan ini sekali atau dua kali sehari. 2. Jahe: Seduh teh jahe yang menenangkan dengan menambahkan 20â€“40 gram irisan jahe segar ke dalam secangkir air panas. Biarkan terendam selama beberapa menit sebelum diminum. Tambahkan madu atau jus lemon untuk meningkatkan rasa dan semakin meredakan batuk. 3. Air: Tetap terhidrasi sangat penting bagi mereka yang menderita batuk atau pilek. Penelitian menunjukkan bahwa meminum air pada suhu ruangan dapat meredakan batuk, pilek, dan bersin.'